## 파인콘 백터 DB 저장

In [26]:
%pip install -qU docx2txt langchain_community langchain-text-splitters langchain-chroma langchain-pinecone  pinecone pinecone 

Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone


## 1. 문서 내용 읽고 분할 #############################################
loader = Docx2txtLoader('law_2.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 벡터 데이터베이스에 저장 ###############################
## 2.1. 환경변수 읽어오기
load_dotenv()

## 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

## 2.3 저장된 파이콘 인덱스 읽어오기
# (cf) 벡터 데이터베이스에 저장
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name='law02-index',
# )

api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

database = PineconeVectorStore(
    index=pc.Index('law02-index'),
    embedding=embedding
)

## 3. 질문 -> 벡터 데이터베이스(vector store)에서 유사도 검색 #############
## 3.1. 사용자 질문
query = '전세사기피해주택의 가격은 얼마인가요?'
# query = '전세사기피해주택 임대인의 국세는 계산은 어떻게 하나요?'
# query ='전세사기피해자는 누구인가요?'
# query = '오늘 점심 메뉴 정해줘!'

## 3.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=2)

## 3.3 문서 객체 2개 -> 하나의 문자열
context = '😃👌\n'.join([doc.page_content for doc in retrieved_docs])

## 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ###################
## 4.1. 프롬프트 작성  
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.
- 전세사기피해 법률 이외의 질문에는 '답변을 할 수 없습니다.'로 답하세요.

[context]
{retrieved_docs}

Question : {query}
'''

## 4.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs=context, 
    query=query,
    )

## 4.3. LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서 보내기  
ai_message = llm.invoke(formatted_prompt)
ai_message.content

'전세사기피해주택의 가격을 구체적으로 설명하려면 여러 요소가 고려되어야 합니다. 전세사기피해주택의 가격은 주택의 위치, 면적, 상태, 시가 표준액 등 다양한 요소에 따라 달라질 수 있습니다. 이와 관련된 구체적인 사항은 감정평가법인등의 감정 평가 및 해당 법령에 따라 산정됩니다. 정확한 평가를 위해서는 전문 감정평가사와 상담하시기를 권장드립니다.'